In [1]:
import pandas as pd
import numpy as np
import datasets

In [29]:
coig_pc_dset = datasets.load_dataset('BAAI/COIG-PC-Lite', split='full')

In [3]:
en_flan_dset = datasets.load_dataset('Muennighoff/flan', split='train')

Resolving data files:   0%|          | 0/132 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/132 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/132 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
fr_instruct_dset = datasets.load_dataset('Enno-Ai/fr-instructs', split='train')

Generating train split:   0%|          | 0/11794112 [00:00<?, ? examples/s]

In [5]:
russian_instruct_dset = datasets.load_dataset('Den4ikAI/russian_instructions_2', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
german_guanaco_dset = datasets.load_dataset('tessiw/German_GuanacoDataset', split='train')
german_evol_instruct_dset = datasets.load_dataset('mayflowergmbh/evol-instruct_de', split='train')

Generating train split:   0%|          | 0/139476 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
portuguese_instruct_dset = datasets.load_dataset('dominguesm/Canarim-Instruct-PTBR-Dataset', split='train')
portuguese_aira_dset = datasets.load_dataset('nicholasKluge/instruct-aira-dataset', split='portuguese')

Generating train split:   0%|          | 0/316413 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1519 [00:00<?, ? examples/s]

Generating portuguese split:   0%|          | 0/40945 [00:00<?, ? examples/s]

Generating english split:   0%|          | 0/41762 [00:00<?, ? examples/s]

Generating spanish split:   0%|          | 0/40946 [00:00<?, ? examples/s]

In [8]:
indic_instruct_dset = datasets.load_dataset('convaiinnovations/Nadi_Indic466k_Instruct', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
tamil_instruct_dset = datasets.load_dataset('Hemanth-thunder/tamil-open-instruct-v1', split='train')

Generating train split:   0%|          | 0/493813 [00:00<?, ? examples/s]

In [10]:
indo_instruct_dset = datasets.load_dataset('indonlp/nusa_t2t', data_files=['train_t2t_data_v1.csv']) 

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/29 [00:00<?, ?it/s]

# Check Dataset

In [11]:
indo_instruct_dset

DatasetDict({
    train: Dataset({
        features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
        num_rows: 30179364
    })
})

In [12]:
tamil_instruct_dset

Dataset({
    features: ['instruction', 'output', 'input', 'text'],
    num_rows: 493813
})

In [13]:
indic_instruct_dset # Too many low-resource languages, we can skip this one

Dataset({
    features: ['instruction', 'response'],
    num_rows: 466257
})

In [14]:
portuguese_aira_dset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 40945
})

In [15]:
portuguese_instruct_dset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 316413
})

In [16]:
german_evol_instruct_dset

Dataset({
    features: ['output', 'history', 'instruction', 'input'],
    num_rows: 59022
})

In [20]:
german_guanaco_dset

Dataset({
    features: ['text'],
    num_rows: 139476
})

In [21]:
russian_instruct_dset

Dataset({
    features: ['question', 'answer'],
    num_rows: 237281
})

In [22]:
fr_instruct_dset

Dataset({
    features: ['instruction', 'input', 'output', 'source'],
    num_rows: 11794112
})

In [23]:
en_flan_dset

Dataset({
    features: ['inputs', 'targets', 'task'],
    num_rows: 2772100
})

In [30]:
coig_pc_dset

Dataset({
    features: ['instruction', 'input', 'output', 'split', 'task_name_in_eng', 'task_type', 'domain', 'other', 'filename'],
    num_rows: 650147
})

# Sample & Standardize Dataset

In [31]:
# Experiment Multi Size: 0, 0.1, 0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0, 5.0
# Multi Data: (655000 * 3) + 237281 + 139476 + 59022 + 316413 + 40945 + 493813 = 3251950
# COIG 5x: 650147 * 5 = 3250735

### Dataset Sampling

In [55]:
def random_sample(dset, n_samples=655000, random_seed=14045):
    return dset.shuffle(seed=random_seed).select(range(n_samples))

In [61]:
s_en_flan_dset = random_sample(en_flan_dset)
s_fr_instruct_dset = random_sample(fr_instruct_dset)
s_indo_instruct_dset = random_sample(indo_instruct_dset['train'])

### Standardize Dataset

In [164]:
def format_coig_pc(row):
    if row['input']:
        row['input'] = f"[INST] {row['instruction']}\n{row['input']} [/INST]"
    else:
        row['input'] = f"[INST] {row['instruction']} [/INST]"
    return row

std_coig_pc_dset = coig_pc_dset.map(format_coig_pc, num_proc=64,
     remove_columns=['instruction', 'split', 'task_name_in_eng', 'task_type', 'domain', 'other', 'filename']
)
std_coig_pc_dset.save_to_disk('./cache/zho_train')

Saving the dataset (0/2 shards):   0%|          | 0/650147 [00:00<?, ? examples/s]

In [165]:
def format_en_flan(row):
    row['inputs'] = f"[INST] {row['inputs']} [/INST]"
    return row

std_en_flan_dset = s_en_flan_dset.map(
    format_en_flan, num_proc=64, remove_columns=['task']
).rename_columns({'inputs': 'input', 'targets': 'output'})
std_en_flan_dset.save_to_disk('./cache/eng_train')

Saving the dataset (0/2 shards):   0%|          | 0/655000 [00:00<?, ? examples/s]

In [166]:
def format_fr_instruct(row):
    if row['input']:
        row['input'] = f"[INST] {row['instruction']}\n{row['input']} [/INST]"
    else:
        row['input'] = f"[INST] {row['instruction']} [/INST]"
    return row
std_fr_instruct_dset = s_fr_instruct_dset.map(format_fr_instruct, num_proc=64, remove_columns=['instruction', 'source'])
std_fr_instruct_dset.save_to_disk('./cache/fra_train')

Saving the dataset (0/1 shards):   0%|          | 0/655000 [00:00<?, ? examples/s]

In [167]:
def format_indo_instruct(row):
    row['input'] = f"[INST] {row['input']} [/INST]"
    row['output'] = "None" if row['output'] is None else row['output']
    return row
std_indo_instruct_dset = s_indo_instruct_dset.map(format_indo_instruct, num_proc=64, remove_columns=[
    'dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key'
])
std_indo_instruct_dset.save_to_disk('./cache/ind_train')

Map (num_proc=64):   0%|          | 0/655000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/655000 [00:00<?, ? examples/s]

In [168]:
def format_russian_instruct(row):
    row['input'] = f"[INST] {row['question']} [/INST]"
    row['output'] = row['answer']
    return row
std_rus_instruct_dset = russian_instruct_dset.map(format_russian_instruct, remove_columns=['question', 'answer'], num_proc=64)
std_rus_instruct_dset.save_to_disk('./cache/rus_train')

Saving the dataset (0/1 shards):   0%|          | 0/237281 [00:00<?, ? examples/s]

In [169]:
def format_german_guanaco(row):
    inputs, outputs = row['text'].replace('<s>','').replace('</s>','').strip().split('[/INST]')
    row['input'] = f"{inputs.strip()} [/INST]"
    row['output'] = outputs
    return row
std_german_guanaco_dset = german_guanaco_dset.map(format_german_guanaco, remove_columns=['text'], num_proc=64)

def format_german_evol(row):
    if row['input']:
        row['input'] = f"[INST] {row['instruction']}\n{row['input']} [/INST]"
    else:
        row['input'] = f"[INST] {row['instruction']} [/INST]"
    return row
std_german_evol_dset = german_evol_instruct_dset.map(format_german_evol, remove_columns=['history', 'instruction'], num_proc=64)

datasets.concatenate_datasets([std_german_guanaco_dset, std_german_evol_dset]).save_to_disk('./cache/deu_train')

Map (num_proc=64):   0%|          | 0/59022 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/198498 [00:00<?, ? examples/s]

In [170]:
def format_tamil_instruct(row):
    if row['input']:
        row['input'] = f"[INST] {row['instruction']}\n{row['input']} [/INST]"
    else:
        row['input'] = f"[INST] {row['instruction']} [/INST]"
    return row
std_tamil_instruct_dset = tamil_instruct_dset.map(format_tamil_instruct, remove_columns=['instruction', 'text'], num_proc=64)
std_tamil_instruct_dset.save_to_disk('./cache/tam_train')

Saving the dataset (0/2 shards):   0%|          | 0/493813 [00:00<?, ? examples/s]

In [171]:
def format_portuguese_aira(row):
    row['input'] = f"[INST] {row['prompt']} [/INST]"
    row['output'] = row['completion']
    return row
std_portuguese_aira_dset = portuguese_aira_dset.map(format_portuguese_aira, remove_columns=['prompt', 'completion'], num_proc=64)

def format_portuguese_instruct(row):
    if row['input']:
        row['input'] = f"[INST] {row['instruction']}\n{row['input']} [/INST]"
    else:
        row['input'] = f"[INST] {row['instruction']} [/INST]"
    return row
std_portuguese_instruct_dset = portuguese_instruct_dset.map(format_tamil_instruct, remove_columns=['instruction'], num_proc=64)
datasets.concatenate_datasets([std_portuguese_aira_dset, std_portuguese_instruct_dset]).save_to_disk('./cache/por_train')

Saving the dataset (0/1 shards):   0%|          | 0/357358 [00:00<?, ? examples/s]